# Modellierung 

In diesem Notebook wird die Modellierungsphase für die Betrugserkennung durchgeführt.
Der Fokus liegt auf einer robusten und reproduzierbaren Evaluierung unter Berücksichtigung
der stark unausgeglichenen Klassenverteilung.

Alle Merkmale wurden bereits im Preprocessing-Schritt bereinigt und transformiert.


In [122]:
# Grundlegende Bibliotheken
import numpy as np
import pandas as pd

# Visualisierung 
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_validate

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    classification_report
)

# XGBoost
from xgboost import XGBClassifier

In [123]:
# Pfad zu den vorverarbeiteten Daten
DATA_PATH = "../data/processed/training_preprocessed.csv"

# Laden der Daten
df = pd.read_csv(DATA_PATH)

print("Shape des Datensatzes:", df.shape)
df.head(10)

Shape des Datensatzes: (95456, 17)


,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,FraudResult,ts_month,ts_is_weekend,ts_is_night,ts_hour_sin,ts_hour_cos,ts_dow_sin,ts_dow_cos,log_value,amount_value_ratio
0,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969,6.908755,0.999001
1,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969,3.044522,0.952381
2,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969,6.216606,0.998004
3,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,9.989711,0.917389
4,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,6.469250,0.998450
5,ProviderId_6,ProductId_3,airtime,ChannelId_3,2000.0,2000,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,7.601402,0.999500
6,ProviderId_5,ProductId_3,airtime,ChannelId_3,10000.0,10000,4,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,9.210440,0.999900
7,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-500.0,500,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,6.216606,0.998004
8,ProviderId_6,ProductId_10,airtime,ChannelId_3,500.0,500,2,0,11,0,1,0.866025,0.500000,0.433884,-0.900969,6.216606,0.998004
9,ProviderId_1,ProductId_15,financial_services,ChannelId_3,600.0,600,2,0,11,0,1,0.866025,0.500000,0.433884,-0.900969,6.398595,0.998336


In [124]:
# Features und Zielvariable trennen
TARGET = "FraudResult"

X = df.drop(columns=[TARGET])
y = df[TARGET]

print("Shape der Feature-Matrix X:", X.shape)
print("Target-Verteilung:")
print(y.value_counts(normalize=True))

Shape der Feature-Matrix X: (95456, 16)
Target-Verteilung:
FraudResult
0    0.997978
1    0.002022
Name: proportion, dtype: float64


## Train-Test-Split

Die Daten werden in Trainings- und Testdaten aufgeteilt.
Dabei wird ein stratifizierter Split verwendet, um die Klassenverteilung
in beiden Datensätzen konstant zu halten.


In [125]:
# Train/Test-Split(stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Trainingsset Shape:", X_train.shape)
print("Testset Shape:", X_test.shape)

print("Trainingsset Target-Verteilung:")
print(y_train.value_counts(normalize=True)*100)

print("Testset Target-Verteilung:")
print(y_test.value_counts(normalize=True)*100)


Trainingsset Shape: (76364, 16)
Testset Shape: (19092, 16)
Trainingsset Target-Verteilung:
FraudResult
0    99.798334
1     0.201666
Name: proportion, dtype: float64
Testset Target-Verteilung:
FraudResult
0    99.795726
1     0.204274
Name: proportion, dtype: float64


## Evaluationsmetriken

Aufgrund der starken Klassenunbalance sind Accuracy-basierte Metriken ungeeignet.
Verwendet werden daher:

- ROC-AUC (Trennschärfe)
- PR-AUC (Performance bei seltenen Klassen)
- Recall (Erkennungsrate von Betrugsfällen)
- F1-Score (Trade-off zwischen Precision und Recall)


In [126]:
# Definition der Metriken
scoring = {
    "roc_auc": "roc_auc",
    "pr_auc": "average_precision",
    "recall": "recall",
    "f1": "f1"

}

## Baseline: Logistic Regression

Als Referenzmodell wird eine Logistic Regression mit
class_weight="**balanced**" trainiert.
Die Evaluation erfolgt mittels stratified Cross-Validation.


In [127]:
# Trennung der numerischen und kategorialen Merkmale
categorical_cols = X_train.select_dtypes(include=["object"]).columns.tolist()
numeric_cols = X_train.select_dtypes(include=["number"]).columns.tolist()

# Preprocessing-Pipeline für numerische Merkmale
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
# Preprocessing-Pipeline für kategoriale Merkmale
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Kombinieren der Preprocessing-Pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)

# Vollständige Pipeline mit Logistic Regression
model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(class_weight="balanced",max_iter=1000, random_state=42))
])

## Cross-Validation mit StratifiedKFold

Zur robusten Bewertung der Modellleistung wird eine stratifizierte Cross-Validation
mit 5 Folds verwendet.

Durch die Stratifikation bleibt die stark unausgeglichene Klassenverteilung
(Betrug vs. Nicht-Betrug) in jedem Fold erhalten.
Dies verhindert verzerrte Schätzungen der Modellgüte.

Die Evaluation erfolgt anhand mehrerer Metriken:
- ROC-AUC
- PR-AUC (Average Precision)
- Recall
- F1-Score



In [128]:
# Cross-Validation mit StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_validate(model_pipeline, X_train, y_train, cv=cv, scoring=scoring)

# Ergebnisse der Cross-Validation anzeigen
# Ergebnisse zusammenfassen
cv_summary = pd.DataFrame({
    "ROC-AUC": cv_results["test_roc_auc"],
    "PR-AUC": cv_results["test_pr_auc"],
    "Recall": cv_results["test_recall"],
    "F1": cv_results["test_f1"]
})

display(cv_summary)

print("\nDurchschnittliche CV-Ergebnisse:")
display(cv_summary.mean())



,ROC-AUC,PR-AUC,Recall,F1
0,0.982701,0.664837,0.967742,0.413793
1,0.997871,0.675407,0.967742,0.338983
2,0.999062,0.640737,1.000000,0.335135
3,0.988924,0.750211,0.967742,0.372671
4,0.998837,0.826339,0.966667,0.345238



Durchschnittliche CV-Ergebnisse:


ROC-AUC    0.993479
PR-AUC     0.711506
Recall     0.973978
F1         0.361164
dtype: float64

## Vergleichsmodell: XGBoost

XGBoost wird aufgrund seiner Fähigkeit gewählt,
nicht-lineare Zusammenhänge und Klassenunbalance effektiv zu modellieren.

Die Klassenunbalance wird über `scale_pos_weight` berücksichtigt.


In [145]:
# Vergleich mit XGBoost
neg,pos = np.bincount(y_train)
scale_pos_weight = neg / pos  

print(f"scale_pos_weight für XGBoost: {scale_pos_weight:.2f}")

scale_pos_weight für XGBoost: 494.87


In [146]:
# Definition des Modells mit XGBoost
xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="aucpr",  # TRÈS important pour fraude
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)


In [147]:
# Pipeline XGBoost(preprocessing bleibt gleich)
xgb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", xgb_model)
])

In [148]:
# Cross-Validation XGBoost(bleibt gleich wie bei Logistic Regression)
xgb_cv_results = cross_validate(xgb_pipeline, X_train, y_train, cv=cv, scoring=scoring)

In [149]:
print("\nXGBoost CV-Ergebnisse:")
xgb_cv_summary = pd.DataFrame({
    "ROC-AUC": xgb_cv_results["test_roc_auc"],
    "PR-AUC": xgb_cv_results["test_pr_auc"],
    "Recall": xgb_cv_results["test_recall"],
    "F1": xgb_cv_results["test_f1"]
})
display(xgb_cv_summary)
print("\nDurchschnittliche CV-Ergebnisse:")
display(cv_summary.mean())



XGBoost CV-Ergebnisse:


,ROC-AUC,PR-AUC,Recall,F1
0,0.991531,0.686807,0.838710,0.584270
1,0.996373,0.619140,0.870968,0.666667
2,0.998928,0.538340,0.967742,0.600000
3,0.994876,0.703279,0.870968,0.600000
4,0.998877,0.602181,0.966667,0.585859



Durchschnittliche CV-Ergebnisse:


ROC-AUC    0.993479
PR-AUC     0.711506
Recall     0.973978
F1         0.361164
dtype: float64

## Hyperparameter-Tuning

Zur Optimierung der Modellleistung wird RandomizedSearchCV eingesetzt.
Als Optimierungsziel wird die Average Precision (PR-AUC) verwendet,
da sie besonders geeignet für Betrugserkennung ist.


In [139]:
# Hyperparameter-Tuning mit RandomizedSearchCV (optional, kann lange dauern)
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

xgb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", XGBClassifier(
        objective="binary:logistic",
        eval_metric="aucpr",   # très important pour fraude
        tree_method="hist",    # stabilité avec OneHot
        random_state=42,
        n_jobs=-1
    ))
])


In [141]:
param_dist = {
    "model__n_estimators": [100, 200, 300],
    "model__max_depth": [3, 5, 7],
    "model__learning_rate": [0.01, 0.05, 0.1],
    "model__subsample": [0.6, 0.8, 1.0],
    "model__colsample_bytree": [0.6, 0.8, 1.0],
    "model__scale_pos_weight": [scale_pos_weight] 
}

In [142]:
random_search = RandomizedSearchCV(
    estimator=xgb_pipeline,
    param_distributions=param_dist,
    n_iter=10,
    scoring="average_precision",  
    cv=cv,                        
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['Amount',
                                                                                'Value',
                                                                                'PricingStrategy',
                                                                                'ts_month',
                                                                                'ts_is_weekend',
                                                                                'ts_is_night',
                                                                                'ts_hour_sin',
                                                                                'ts_hour...
                                                            n_estimators=None,
                                                            n_jobs=-1,
                                                            num_parallel_tree=None, ...))]),
                   n_jobs=-1,
                   param_distributions={'model__colsample_bytree': [0.6, 0.8,
                                                                    1.0],
                                        'model__learning_rate': [0.01, 0.05,
                                                                 0.1],
                                        'model__max_depth': [3, 5, 7],
                                        'model__n_estimators': [100, 200, 300],
                                        'model__scale_pos_weight': [494.87012987012986],
                                        'model__subsample': [0.6, 0.8, 1.0]},
                   random_state=42, scoring='average_precision', verbose=2)

In [143]:
print("Beste Hyperparameter:", random_search.best_params_)

Beste Hyperparameter: {'model__subsample': 0.8, 'model__scale_pos_weight': 494.87012987012986, 'model__n_estimators': 200, 'model__max_depth': 7, 'model__learning_rate': 0.1, 'model__colsample_bytree': 1.0}


In [144]:
print("XGBoost TUNED CV-Ergebnisse:")
tuned_xgb_cv_results = cross_validate(random_search.best_estimator_, X_train, y_train, cv=cv, scoring=scoring)
tuned_xgb_cv_summary = pd.DataFrame({
    "ROC-AUC": tuned_xgb_cv_results["test_roc_auc"],
    "PR-AUC": tuned_xgb_cv_results["test_pr_auc"],
    "Recall": tuned_xgb_cv_results["test_recall"],
    "F1": tuned_xgb_cv_results["test_f1"]
})
display(tuned_xgb_cv_summary)
print("\nDurchschnittliche CV-Ergebnisse:")
display(tuned_xgb_cv_summary.mean())


XGBoost TUNED CV-Ergebnisse:


,ROC-AUC,PR-AUC,Recall,F1
0,0.987840,0.684655,0.709677,0.586667
1,0.985999,0.681221,0.677419,0.636364
2,0.998723,0.540335,0.806452,0.625000
3,0.998248,0.642615,0.806452,0.625000
4,0.999192,0.693854,0.900000,0.627907



Durchschnittliche CV-Ergebnisse:


ROC-AUC    0.994000
PR-AUC     0.648536
Recall     0.780000
F1         0.620187
dtype: float64